In [1]:
pip install tensorflow-hub pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.3/90.3 kB 5.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [29]:
import numpy as np
from PIL import Image
import tensorflow as tf
import tensorflow_hub as hub
import warnings

warnings.filterwarnings('ignore')

**Image Preprocessing**

The preprocess_image function performs preprocessing on a single image for it to be used by the model. It first opens an image at a given path and then resizes the image to the target size that the model requires. It then converts the image data to a NumPy array and normalizes pixel values to be within the range [0,1]. Finally, because the model expects inputs to be batches of images, we add an extra dimension to the image tensor. Even though we are only making a prediction on one image at a time, it needs to be treated as part of a batch.

In [39]:
def preprocess_image(image_path, target_size=(224, 224)):
    # Open the image file
    img = Image.open(image_path)
    
    # Resize the image to the target size. 
    # This is necessary because the model expects inputs to be of a certain size.
    img = img.resize(target_size)
    
    # Convert the image data to a NumPy array and normalize pixel values to [0, 1]
    # Image data is often represented as integers in the range [0, 255].
    # However, many models expect the input data to be in the range [0, 1], hence we divide by 255.0
    img = np.array(img) / 255.0
    
    # Add an extra dimension to the image tensor
    # This is necessary because the model expects batches of images as input.
    # Even though we're only predicting on one image at a time, it still has to be part of a batch.
    img = np.expand_dims(img, axis=0)
    
    return img

**Defining the EfficientnetB0 Model**

This chunk shows how to utilize a pre-trained model, specifically the EfficientNetB0 model, using TensorFlow Hub. The model is downloaded from the provided URL and incorporated into our new Sequential model as a Keras Layer. The 'trainable' parameter is set to False, indicating that the weights of the pre-trained model are frozen and won't be updated during training. This technique is often referred to as 'Transfer Learning', which leverages the knowledge gained from training a model on a large dataset, in this case, the ImageNet dataset, and applying it to another related problem.

In [31]:
# Define the URL where the pre-trained model can be downloaded
model_url = "https://tfhub.dev/tensorflow/efficientnet/b0/classification/1"

# Create a new Sequential model
model = tf.keras.Sequential([
    # Add a Keras Layer which uses the pre-trained model from TensorFlow Hub
    # We set trainable to False to freeze the weights of the pre-trained model
    hub.KerasLayer(model_url, trainable=False)
])

2023-07-14 23:18:54.394207: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype int32 and shape [4]
	 [[{{node inputs}}]]
2023-07-14 23:18:54.394347: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype int32 and shape [4]
	 [[{{node inputs}}]]
2023-07-14 23:18:56.931258: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype int32 and shape [4]
	 [[{{node inputs}}]]
2023-07-14 23:18:56.931441: 

**Making a Prediction**

This chunk illustrates the process of using a trained model to make predictions on a specific image. First, the path to the image is defined. The image is then preprocessed, using the previously defined preprocess_image function, to meet the requirements of the model. The model then predicts the class probabilities for the preprocessed image. A helper function from the keras EfficientNet module, decode_predictions, is used to convert the class probabilities into human-readable class labels. The top 5 predictions, including their class names and probabilities, are then printed out for review.

In [42]:
# Define the path to the image we want to make predictions on
image_path = '/home/jovyan/shared/jiarui-notebooks/main-qimg-a0299f9fe9ff1e7e756a66901d3e6fb0-lq.jpeg'

# Preprocess the image using our predefined function
img = preprocess_image(image_path)

# Use the model to predict the class probabilities of the preprocessed image
predictions = model.predict(img)

# Define a helper function to decode the predictions into human-readable class names
decode_predictions = tf.keras.applications.efficientnet.decode_predictions

# Use the helper function to decode the top 5 predictions
predicted_classes = decode_predictions(predictions, top=5)

# Print out the top 5 predictions along with their probabilities
for i, (imagenet_id, name, prob) in enumerate(predicted_classes[0]):
    print(f"{i+1}: {name} ({imagenet_id}) - {prob*100:.2f}%")

(1, 224, 224, 3)
1/1 [==============================] - 0s 88ms/step
1: tench (n01440764) - 43.42%
2: barracouta (n02514041) - 11.98%
3: gar (n02641379) - 6.03%
4: coho (n02536864) - 5.44%
5: reel (n04067472) - 1.94%
